In [2]:
import pandas as pd
from textblob import TextBlob
import nltk
from textblob import Word
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split 
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing

In [3]:
df=pd.read_csv("/Users/aman/coding stuff/sem 6/tarp/dataset/Suicide_preprocessed1.csv")
#df = pd.read_csv( "/Users/aman/coding stuff/sem 6/tarp/dataset/Suicide_preprocessed1.csv", index_col=None, header=0, engine='python' )

In [4]:
df.head(10)

,Unnamed: 0,text,class
0,0,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,1,Am I weird I don t get affected by compliments...,non-suicide
2,2,Finally is almost over So I can never hear has...,non-suicide
3,3,i need helpjust help me im crying so hard,suicide
4,4,I m so lostHello my name is Adam and I ve been...,suicide
5,5,Honetly idkI dont know what im even doing here...,suicide
6,6,Trigger warning Excuse for self inflicted bur...,suicide
7,7,It ends tonight I can t do it anymore I quit,suicide
8,8,Everyone wants to be edgy and it s making me s...,non-suicide
9,9,My life is over at years oldHello all I am a y...,suicide


In [5]:
l=[]
for i in range(10000):
    l.append(TextBlob(df.iloc[i,1]).sentiment.polarity)
l1=[]
for i in l:
    if i>=0.2:
        l1.append("non-suicide")
    else:
        l1.append("suicide")
l2=df.iloc[0:10000,2].tolist()


In [6]:
from sklearn.metrics import accuracy_score
print(accuracy_score(l1,l2))

0.5704


In [7]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /Users/aman/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/aman/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [5]:
def cleaning(df):
    df['text'] = df['text'].apply(lambda x: ' '.join(x.lower() for x in x.split()))
    # Lemmatization
    df['text'] = df['text'].apply(lambda x: ' '.join([Word(x).lemmatize() for x in x.split()]))
    return df

In [9]:

#Generating Embeddings using tokenizer
df=df.iloc[0:100,:]
data_cleaned=cleaning(df)
tokenizer = Tokenizer(num_words=500, split=' ') 
tokenizer.fit_on_texts(data_cleaned['text'].values)
X = tokenizer.texts_to_sequences(data_cleaned['text'].values)
X = pad_sequences(X)
#Model Building
model = Sequential()
model.add(Embedding(500, 120, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(704, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(352, activation='LeakyReLU'))
model.add(Dense(3, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())
df["class"]=preprocessing.LabelEncoder().fit_transform(df["class"])
X_train,X_test,y_train,y_test=train_test_split(X,df["class"],test_size=0.2)

y_train = to_categorical(y_train, 3)
y_test = to_categorical(y_test, 3)
#Model Training
print(X_train,X_test,y_train,y_test)
model.fit(X_train, y_train, epochs = 20, batch_size=32, verbose =1)
#Model Testing
model.evaluate(X_test,y_test)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1580, 120)         60000     
                                                                 
 spatial_dropout1d (SpatialD  (None, 1580, 120)        0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 704)               2323200   
                                                                 
 dense (Dense)               (None, 352)               248160    
                                                                 
 dense_1 (Dense)             (None, 3)                 1059      
                                                                 
Total params: 2,632,419
Trainable params: 2,632,419
Non-trainable params: 0
______________________________________________

2023-02-08 21:08:26.346355: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


3/3 [==============================] - 51s 16s/step - loss: 1.0663 - accuracy: 0.5500
Epoch 2/20
3/3 [==============================] - 49s 16s/step - loss: 1.0082 - accuracy: 0.5375
Epoch 3/20
3/3 [==============================] - 51s 17s/step - loss: 0.9237 - accuracy: 0.3875
Epoch 4/20
3/3 [==============================] - 49s 16s/step - loss: 0.8828 - accuracy: 0.6875
Epoch 5/20
3/3 [==============================] - 51s 17s/step - loss: 0.7521 - accuracy: 0.6375
Epoch 6/20
3/3 [==============================] - 49s 16s/step - loss: 0.6708 - accuracy: 0.6125
Epoch 7/20
3/3 [==============================] - 48s 16s/step - loss: 0.6802 - accuracy: 0.6125
Epoch 8/20
3/3 [==============================] - 49s 16s/step - loss: 0.7000 - accuracy: 0.4625
Epoch 9/20
3/3 [==============================] - 48s 16s/step - loss: 0.7174 - accuracy: 0.6125
Epoch 10/20
3/3 [==============================] - 48s 16s/step - loss: 0.6315 - accuracy: 0.7000
Epoch 11/20
3/3 [=======================

[0.8613475561141968, 0.6000000238418579]

In [10]:
model.evaluate(X_test,y_test)

1/1 [==============================] - 2s 2s/step - loss: 0.8613 - accuracy: 0.6000


[0.8613475561141968, 0.6000000238418579]

In [11]:
print(X_train,len(X_train))
print(X_test,len(X_test))
print(y_train,len(y_train))
print(y_test,len(y_test))

[[  0   0   0 ...  10   1  18]
 [  0   0   0 ...   2  32   6]
 [  0   0   0 ... 403 306 401]
 ...
 [  0   0   0 ...   2 227 112]
 [  0   0   0 ...  99   1 455]
 [  0   0   0 ... 280   1  32]] 80
[[  0   0   0 ...  13   5 240]
 [  0   0   0 ...   2  15   6]
 [  0   0   0 ...   7  20 101]
 ...
 [  0   0   0 ...  25   5  75]
 [  0   0   0 ... 229  22  23]
 [  0   0   0 ...   0 127  19]] 20
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.

In [ ]:
df_text=df["text"].tolist()
df_class=df["class"].tolist()
print(df_text[0:100])

In [6]:
from transformers import pipeline
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
sentiment_pipeline = pipeline("sentiment-analysis",model=model, tokenizer=tokenizer)
l1=[]
for i in df_text:
    l1.append(sentiment_pipeline(list(i)))
#print(l1)
#print(l1[0]["label"])


KeyboardInterrupt: 

In [14]:
df.head(10)

,Unnamed: 0,text,class
0,0,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,1,Am I weird I don t get affected by compliments...,non-suicide
2,2,Finally is almost over So I can never hear has...,non-suicide
3,3,i need helpjust help me im crying so hard,suicide
4,4,I m so lostHello my name is Adam and I ve been...,suicide
5,5,Honetly idkI dont know what im even doing here...,suicide
6,6,Trigger warning Excuse for self inflicted bur...,suicide
7,7,It ends tonight I can t do it anymore I quit,suicide
8,8,Everyone wants to be edgy and it s making me s...,non-suicide
9,9,My life is over at years oldHello all I am a y...,suicide


In [ ]:
l3=[]
for i in l1:
    if i["label"]=="NEGATIVE":
        l3.append("suicide")
    else:
        l3.append("non-suicide")


['suicide', 'non-suicide', 'suicide', 'suicide', 'suicide', 'suicide', 'suicide', 'suicide', 'suicide', 'suicide', 'suicide', 'suicide', 'suicide', 'suicide', 'suicide', 'suicide', 'suicide', 'suicide', 'suicide', 'suicide', 'suicide', 'suicide']


In [ ]:
l2=df_class
print(accuracy_score(l2,l3))

0.7272727272727273
